In [10]:
!pip install num2words
!pip install nltk
!pip install pip beautifulsoup4
!pip install pip WordCloud
!pip install pip Counter
!pip install pip Word2Vec
!pip install scikit-learn
!pip install gensim
!pip install keras
!pip install tensorflow
#==2.4.1 keras==2.4.3
!pip install keras-preprocessing

In [13]:
import pandas as pd
import unicodedata
from num2words import num2words
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from collections import Counter
from sklearn.manifold import TSNE
from gensim.models import Word2Vec

import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk import ngrams

from collections import Counter
import re



nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

nltk_sw_list = stopwords.words('english')
nltk_lematizer_list = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [14]:
# Funcion para Preprocesado


def eliminar_stop_words(texto):
    tokens = word_tokenize(texto)
    tokens = [word for word in tokens if word not in nltk_sw_list]
    return tokens

def eliminar_signos_puntuacion(texto):
    text = re.sub(r'[^a-z\s]', '', texto)
    return text

def eliminar_casos_especiales(texto):
    texto= texto.replace('<br />',' ')
    return texto

def elimimnar_etiquetas_html(texto):
   text = BeautifulSoup(texto, "html.parser").get_text()
   texto = re.sub(r'<.*?>', '', texto)
   return texto

def revision_generica(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = text.lower()
    return text


def revision_lematizer(tokens):
    tokens = [nltk_lematizer_list.lemmatize(word) for word in tokens]
    return tokens


def texto_preproceado(text):
    clean_text = list()

    text = revision_generica(text)
    text = elimimnar_etiquetas_html( text)
    text = eliminar_signos_puntuacion(text)
    text = eliminar_casos_especiales(text)
    tokens = eliminar_stop_words(text)
    tokens = revision_lematizer(tokens)

    clean_text = ' '.join(tokens)

    return clean_text

In [15]:


# Inicializamos listas para almacenar registros
registros_1 = []
registros_2 = []
registros_3 = []
registros_4 = []
registros_5 = []

# Especifico la ruta del archivo descomprimido
#https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/All_Beauty.jsonl.gz
ruta_destino_descomprimido = "All_Beauty.jsonl"


# Ejemplo de estructura JSON
#data =
#    {
#        "rating": 4.0,
#        "title": "Works great but smells a little weird.",
#        "text": "This product does what I need it to do, I just wish it was odorless or had a soft coconut smell. Having my head smell like an orange coffee is offputting. (granted, I did know the smell was described but I was hoping it would be light)",
#        "images": [],
#        "asin": "B081TJ8YS3",
#        "parent_asin": "B081TJ8YS3",
#        "user_id": "AGKHLEW2SOWHNMFQIJGBECAF7INQ",
#        "timestamp": 1588615855070,
#        "helpful_vote": 1,
#        "verified_purchase": True
#    },


# Máximo número de registros por categoría
max_registros = 1000
minima_cantida_texto=5



oraciones_totales = []

with open(ruta_destino_descomprimido, 'r') as archivo:
    # Continúa leyendo mientras no se haya alcanzado el máximo en alguna de las listas
    while len(registros_1)<=max_registros  and len(registros_2)<=max_registros   and len(registros_3)<=max_registros   and len(registros_4)<=max_registros  and len(registros_5)<=max_registros :

        linea = archivo.readline()

        # Si se alcanza el final del archivo, detener el bucle
        if not linea:
            break
        registro = pd.read_json(linea, typ='series')
        calificacion = registro['rating']
        titulo=registro['title']
        texto=registro['text']

        texto_normalizado=texto_preproceado(titulo+' '+texto )
        registro['texto_completo']=texto_normalizado
        #oraciones_totales.append(texto_normalizado)

        if calificacion == 1 and len(registros_1)<=max_registros and len(titulo)>minima_cantida_texto and len(texto)>minima_cantida_texto:
          registros_1.append(registro)
        if calificacion == 2 and len(registros_1)<=max_registros and len(titulo)>minima_cantida_texto and len(texto)>minima_cantida_texto:
          registros_2.append(registro)
        if calificacion == 3 and len(registros_1)<=max_registros and len(titulo)>minima_cantida_texto and len(texto)>minima_cantida_texto:
          registros_3.append(registro)
        if calificacion == 4 and len(registros_1)<=max_registros and len(titulo)>minima_cantida_texto and len(texto)>minima_cantida_texto:
          registros_4.append(registro)
        if calificacion == 5 and len(registros_1)<=max_registros and len(titulo)>minima_cantida_texto and len(texto)>minima_cantida_texto:
          registros_5.append(registro)




# Combina las listas en un DataFrame
df_registros_1 = pd.DataFrame(registros_1)
df_registros_1['rating_label']=0   #calificacion negativa
df_registros_2 = pd.DataFrame(registros_2)
df_registros_2['rating_label']=0  #calificacion negativa
df_registros_3 = pd.DataFrame(registros_3)
df_registros_3['rating_label']=0 #calificacion positiva
df_registros_4 = pd.DataFrame(registros_4)
df_registros_4['rating_label']=1  #calificacion positiva
df_registros_5 = pd.DataFrame(registros_5)
df_registros_5['rating_label']=1 #calificacion positiva

# se utilizarán los comentarios calificados con 1, 2, 3 como negativos y 4, 5 Como positivos
df_registros_positivos = pd.concat([df_registros_4, df_registros_5]).reset_index(drop=True)
df_registros_negativos = pd.concat([df_registros_1, df_registros_2,df_registros_3]).reset_index(drop=True)
df_registros_todos= pd.concat([df_registros_1, df_registros_2,df_registros_3,df_registros_4,df_registros_5]).reset_index(drop=True)


<ipython-input-15-e0336778430c>:46: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  registro = pd.read_json(linea, typ='series')
<ipython-input-15-e0336778430c>:46: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  registro = pd.read_json(linea, typ='series')
<ipython-input-15-e0336778430c>:46: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  registro = pd.read_json(linea, typ='series')
<ipython-input-15-e0336778430c>:46: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  registro = pd.read_json(linea, typ='s

In [16]:
# Configuración para que pandas muestre más filas o columnas
pd.set_option('display.max_rows', 20)  # Cambia el límite de filas mostradas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas

# Imprimir con formato tabular
#print(df_registros_todos.to_string(index=False))

print(df_registros_todos[['rating', 'rating_label' ,'texto_completo']])

      rating  rating_label                                     texto_completo
0        1.0             0                   synthetic feeling felt synthetic
1        1.0             0  claim nothing special unfortunately waited lon...
2        1.0             0  halo hair extension halo hair extension simply...
3        1.0             0  dont bother normally like nyx product one leaf...
4        1.0             0  waste money plastic bend matching brow lay ski...
...      ...           ...                                                ...
1735     5.0             1  higher powered uvled curer girl mom nail start...
1736     5.0             1  used today face soft doesnt feel greasy feel m...
1737     5.0             1  loving company product trying several differen...
1738     5.0             1  pleasure using face serum smell darn good enjo...
1739     5.0             1  girl christmas wish list happy several didnt c...

[1740 rows x 3 columns]


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar los datos preprocesados
# Supongamos que tenemos un DataFrame llamado `df` con columnas 'review' (texto) y 'sentiment' (positivo=1, negativo=0)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
      df_registros_todos['texto_completo'],
      df_registros_todos['rating_label'],
      test_size=0.2,
      random_state=42,
      stratify=df_registros_todos['rating_label'])

print("Tamaño del conjunto de entrenamiento:", len(X_train))
print("Tamaño del conjunto de prueba:", len(X_test))

Tamaño del conjunto de entrenamiento: 1392
Tamaño del conjunto de prueba: 348


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(
 max_df=0.85,         # Filtra palabras presentes en más del 85% de los documentos
    min_df=5,            # Filtra palabras que aparecen en menos de 5 documentos
    max_features=2000,   # Limita el vocabulario a las 2000 palabras más importantes
    strip_accents='ascii', # Normaliza eliminando acentos
    ngram_range=(1, 2),  # Incluye unigramas y bigramas
    stop_words='english' # Elimina palabras comunes en inglés
)
cv.fit(X_train)


# Ajustar y transformar los datos
X_train_bow = cv.fit_transform(X_train)
X_test_bow = cv.transform(X_test)

print("Dimensiones de la matriz de entrenamiento:", X_train_bow.shape)

Dimensiones de la matriz de entrenamiento: (1392, 2000)


In [19]:
print(list(cv.vocabulary_.items())[:20])
print(len(cv.vocabulary_))

[('great', 695), ('new', 1120), ('head', 764), ('price', 1299), ('includes', 837), ('hair', 721), ('real', 1378), ('synthetic', 1731), ('doesnt', 448), ('curl', 376), ('good', 681), ('like', 934), ('trying', 1814), ('style', 1702), ('cutting', 385), ('come', 296), ('clip', 271), ('rubber', 1473), ('band', 95), ('great price', 705)]
2000


In [20]:
#Modelo1 : LogisticRegression

from sklearn.linear_model import LogisticRegression

# Entrenar el modelo de Regresión Logística
model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train_bow, y_train)

# Predicciones
y_pred_lr = model_lr.predict(X_test_bow)

In [35]:
#Modelo1 : LogisticRegression  evaluacion
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

accuracy = accuracy_score(y_test, y_pred_lr)
precision = precision_score(y_test, y_pred_lr, average='weighted')  # Usa 'binary' si es binaria
recall = recall_score(y_test, y_pred_lr, average='weighted')
f1 = f1_score(y_test, y_pred_lr, average='weighted')
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Accuracy: 0.78
Precision: 0.80
Recall: 0.78
F1 Score: 0.71


In [22]:
#Modelo2 : Modelo LSTM - revision de ancho de texto a analizar
df_registros_todos['longitud'] = df_registros_todos['texto_completo'].apply(len)

# Calcular estadísticas básicas
estadisticas = {
    'longitud_minima': df_registros_todos['longitud'].min(),
    'longitud_maxima': df_registros_todos['longitud'].max(),
    'longitud_promedio': df_registros_todos['longitud'].mean(),
    'longitud_mediana': df_registros_todos['longitud'].median(),
    'desviacion_estandar': df_registros_todos['longitud'].std()
}

# Convertir estadísticas a un DataFrame para mejor visualización (opcional)
estadisticas_df = pd.DataFrame([estadisticas])

# Mostrar las estadísticas
print(estadisticas_df)

   longitud_minima  longitud_maxima  longitud_promedio  longitud_mediana  \
0               12             7068         316.152874             197.0   

   desviacion_estandar  
0           408.520727  


In [24]:
#Modelo2 : Modelo LSTM - Tokenización y Padding:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Configuración
max_words = 2000  # Considerar las 2000 palabras más frecuentes
max_len = 316     # Longitud máxima de las secuencias

# Tokenización usando CountVectorizer
vectorizer = CountVectorizer(max_features=max_words, token_pattern=r"(?u)\b\w+\b")
vectorizer.fit(X_train)

# Convertir texto a secuencias de índices de palabras
X_train_seq = [list(map(lambda x: vectorizer.vocabulary_.get(x, 0), text.split())) for text in X_train]
X_test_seq = [list(map(lambda x: vectorizer.vocabulary_.get(x, 0), text.split())) for text in X_test]

# Función para padding manual
def pad_sequences_manual(sequences, maxlen, padding="post", truncating="post"):
    padded = np.zeros((len(sequences), maxlen), dtype=np.int32)
    for i, seq in enumerate(sequences):
        if len(seq) > maxlen:
            if truncating == "post":
                seq = seq[:maxlen]
            elif truncating == "pre":
                seq = seq[-maxlen:]
        if padding == "post":
            padded[i, :len(seq)] = seq
        elif padding == "pre":
            padded[i, -len(seq):] = seq
    return padded

# Padding
X_train_pad = pad_sequences_manual(X_train_seq, max_len)
X_test_pad = pad_sequences_manual(X_test_seq, max_len)

# Resultados
print("Dimensiones del conjunto de entrenamiento:", X_train_pad.shape)
print("Dimensiones del conjunto de prueba:", X_test_pad.shape)


Dimensiones del conjunto de entrenamiento: (1392, 316)
Dimensiones del conjunto de prueba: (348, 316)


In [30]:
#Modelo2 : Modelo LSTM - Configuración del Modelo LSTM
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# Crear el modelo
model = Sequential()

# Capa de embeddings
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))

# Capa LSTM
model.add(LSTM(256, dropout=0.3, recurrent_dropout=0.3))

# Capa de salida
model.add(Dense(1, activation='sigmoid'))  # Salida binaria (positivo/negativo)

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [31]:
# #Modelo2 : Modelo LSTM -  Entrenar el modelo
history = model.fit(
    X_train_pad, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.7481 - loss: 0.5810 - val_accuracy: 0.7634 - val_loss: 0.5556
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.7385 - loss: 0.5766 - val_accuracy: 0.7634 - val_loss: 0.5498
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.7569 - loss: 0.5595 - val_accuracy: 0.7634 - val_loss: 0.5617
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.7673 - loss: 0.5496 - val_accuracy: 0.7634 - val_loss: 0.5509
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.7684 - loss: 0.5504 - val_accuracy: 0.7634 - val_loss: 0.5493
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.7569 - loss: 0.5533 - val_accuracy: 0.7634 - val_loss: 0.5522
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.7520 - loss: 0.5664 - val_accuracy: 0.7634 - val_loss: 0.5526
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.7529 - loss: 0.5651 - val_accuracy: 0.7634 - val_loss:

In [33]:
# Modelo2 : Modelo LSTM -  Evaluación en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test_pad, y_test, verbose=1)
print(f"Pérdida en prueba: {test_loss}")
print(f"Acurrancy: {test_acc}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 273ms/step - accuracy: 0.7607 - loss: 0.5478
Pérdida en prueba: 0.5483393669128418
Acurrancy: 0.7614942789077759
